# Decision Tree

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as stats

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, \
cross_val_score, RandomizedSearchCV, cross_validate

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('../data/cleaned_data.csv')

In [ ]:
df.info()

In [ ]:
y = df.target
X = df.drop(columns =['target', 'Series_Complete_Pop_Pct','County_with_State', 'Date'], axis=1) 

In [ ]:
y.value_counts(normalize=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [ ]:
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train, y_train)

In [ ]:
dt.score(X_train, y_train)

In [ ]:
dt.score(X_test, y_test)

In [ ]:
plot_tree(dt);

## Grid Search DT

In [ ]:
grid = {'max_depth': [None, 4, 5, 6],
       'min_samples_split': [1, 2, 3],
       'min_samples_leaf': [1, 9, 10],
        
       }
grid

In [ ]:
gs = GridSearchCV(estimator=dt, scoring='precision', param_grid=grid, verbose=3, n_jobs=-3)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
final_dt = gs.best_estimator_
final_dt

In [ ]:
cross_validate(final_dt, X_train, y_train, scoring=['precision', 'accuracy'],return_train_score=True)

# Logistic Regression